In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from gsm import GaussianSoftmaxModel
from rsm import RecurrentStickbreakingModel
from evaluation import validate, print_flat_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:',''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
if config.model == 'gsm':
    Model = GaussianSoftmaxModel
elif config.model == 'rsm':
    Model = RecurrentStickbreakingModel
model = Model(config)    
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topics_freq_tokens = [[idx_to_word[idx] for idx in topic_freq_idxs] for topic_freq_idxs in topics_freq_idxs]
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_flat_topic_sample(sess, model, topics_freq_tokens=topics_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = Model(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

display(log_df)
print_flat_topic_sample(sess, model, topics_freq_tokens=topics_freq_tokens)

TRAIN:                           VALID:                \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   PPL     NLL   
5000     41   35   99  524.38  859  521.11  3.17  0.10  951.16  1290  945.46   
10000    37   71   59  523.05  844  519.31  3.67  0.07  943.49  1201  937.34   
15000    37  107   19  522.45  835  518.48  3.91  0.06  942.16  1171  935.71   
20000    35  142  119  521.69  829  517.60  4.04  0.05  945.01  1205  938.45   
25000    35  178   79  521.23  825  517.06  4.12  0.05  943.53  1188  937.23   
30000    35  214   39  521.05  822  516.83  4.18  0.04  943.02  1176  936.67   
35000    35  249  139  520.86  819  516.58  4.24  0.04  943.53  1181  937.11   
40000    35  285   99  520.55  817  516.24  4.28  0.04  948.70  1235  942.34   
45000    36  321   59  520.41  815  516.06  4.31  0.03  943.63  1188  937.29   
50000    37  357   19  520.33  813  515.96  4.34  0.03  942.78  1164  936.26   
55000    36  392  119  520.14  812  515.75  4.36  0.03  945.37  1197  938.87   
60000    35  428   79  520.01  810  515.60  4.37  0.03  943.58  1183  937.24   
65000    36  464   39  519.96  809  515.54  4.39  0.03  943.08  1177  936.73   
70000    35  499  139  519.91  809  515.47  4.41  0.03  943.86  1186  937.58   
75000    36  535   99  519.79  808  515.34  4.42  0.03  947.48  1221  941.11   
80000    39  571   59  519.74  807  515.28  4.43  0.02  944.95  1192  938.68   
85000    37  607   19  519.72  806  515.25  4.44  0.02  943.46  1169  937.13   
90000    37  642  119  519.64  806  515.15  4.45  0.02  945.36  1195  938.97   
95000    38  678   79  519.57  805  515.08  4.45  0.02  944.81  1189  938.44   
100000   38  714   39  519.57  805  515.07  4.46  0.02  944.42  1183  938.01   
105000   38  749  139  519.56  804  515.05  4.47  0.02  945.80  1195  939.31   
110000   37  785   99  519.50  804  514.98  4.47  0.02  948.87  1231  942.39   
115000   37  821   59  519.49  804  514.96  4.48  0.02  944.95  1190  938.54   
120000   37  857   19  519.49  803  514.96  4.48  0.02  944.45  1181  938.07   
125000   38  892  119  519.44  803  514.91  4.49  0.02  946.41  1194  939.94   
130000   36  928   79  519.41  803  514.86  4.49  0.02  945.25  1193  938.77   
135000   36  964   39  519.41  803  514.85  4.49  0.02  944.40  1183  937.88   
140000   36  999  139  519.40  802  514.83  4.50  0.02  945.38  1183  938.73   

                     TEST:       
          KL   REG    LOSS  PPL  
5000    5.64  0.05  583.62  961  
10000   6.11  0.04  580.89  935  
15000   6.43  0.03  583.54  955  
20000   6.53  0.02  583.54  955  
25000   6.28  0.02  583.54  955  
30000   6.32  0.02  583.54  955  
35000   6.40  0.02  583.54  955  
40000   6.34  0.02  583.54  955  
45000   6.33  0.02  583.54  955  
50000   6.51  0.01  584.21  961  
55000   6.48  0.02  584.21  961  
60000   6.33  0.01  584.21  961  
65000   6.33  0.01  584.21  961  
70000   6.26  0.01  584.21  961  
75000   6.36  0.01  584.21  961  
80000   6.25  0.01  584.21  961  
85000   6.33  0.01  584.21  961  
90000   6.38  0.01  584.21  961  
95000   6.36  0.01  584.21  961  
100000  6.39  0.01  584.21  961  
105000  6.48  0.01  584.21  961  
110000  6.47  0.01  584.21  961  
115000  6.40  0.01  584.21  961  
120000  6.36  0.01  584.21  961  
125000  6.46  0.01  584.21  961  
130000  6.47  0.01  584.21  961  
135000  6.51  0.01  584.21  961  
140000  6.64  0.01  584.21  961

0 use file get drive program write window system work one
1 write get article go like think know one say see
2 one people write say think make know post article point
3 god say one people believe christian jesus write know make
4 key space use encryption launch chip system government clipper nasa
5 car bike dod ride get one engine good article write
6 use ground wire circuit power one current amp wiring outlet
7 game year team win player run hit play last season
8 team game player play hockey season go nhl win year
9 pt game play team period la hockey vs goal van


TRAIN:                           VALID:                \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   PPL     NLL   
5000     41   35   99  524.38  859  521.11  3.17  0.10  951.16  1290  945.46   
10000    37   71   59  523.05  844  519.31  3.67  0.07  943.49  1201  937.34   
15000    37  107   19  522.45  835  518.48  3.91  0.06  942.16  1171  935.71   
20000    35  142  119  521.69  829  517.60  4.04  0.05  945.01  1205  938.45   
25000    35  178   79  521.23  825  517.06  4.12  0.05  943.53  1188  937.23   
30000    35  214   39  521.05  822  516.83  4.18  0.04  943.02  1176  936.67   
35000    35  249  139  520.86  819  516.58  4.24  0.04  943.53  1181  937.11   
40000    35  285   99  520.55  817  516.24  4.28  0.04  948.70  1235  942.34   
45000    36  321   59  520.41  815  516.06  4.31  0.03  943.63  1188  937.29   
50000    37  357   19  520.33  813  515.96  4.34  0.03  942.78  1164  936.26   
55000    36  392  119  520.14  812  515.75  4.36  0.03  945.37  1197  938.87   
60000    35  428   79  520.01  810  515.60  4.37  0.03  943.58  1183  937.24   
65000    36  464   39  519.96  809  515.54  4.39  0.03  943.08  1177  936.73   
70000    35  499  139  519.91  809  515.47  4.41  0.03  943.86  1186  937.58   
75000    36  535   99  519.79  808  515.34  4.42  0.03  947.48  1221  941.11   
80000    39  571   59  519.74  807  515.28  4.43  0.02  944.95  1192  938.68   
85000    37  607   19  519.72  806  515.25  4.44  0.02  943.46  1169  937.13   
90000    37  642  119  519.64  806  515.15  4.45  0.02  945.36  1195  938.97   
95000    38  678   79  519.57  805  515.08  4.45  0.02  944.81  1189  938.44   
100000   38  714   39  519.57  805  515.07  4.46  0.02  944.42  1183  938.01   
105000   38  749  139  519.56  804  515.05  4.47  0.02  945.80  1195  939.31   
110000   37  785   99  519.50  804  514.98  4.47  0.02  948.87  1231  942.39   
115000   37  821   59  519.49  804  514.96  4.48  0.02  944.95  1190  938.54   
120000   37  857   19  519.49  803  514.96  4.48  0.02  944.45  1181  938.07   
125000   38  892  119  519.44  803  514.91  4.49  0.02  946.41  1194  939.94   
130000   36  928   79  519.41  803  514.86  4.49  0.02  945.25  1193  938.77   
135000   36  964   39  519.41  803  514.85  4.49  0.02  944.40  1183  937.88   
140000   36  999  139  519.40  802  514.83  4.50  0.02  945.38  1183  938.73   

                     TEST:       
          KL   REG    LOSS  PPL  
5000    5.64  0.05  583.62  961  
10000   6.11  0.04  580.89  935  
15000   6.43  0.03  583.54  955  
20000   6.53  0.02  583.54  955  
25000   6.28  0.02  583.54  955  
30000   6.32  0.02  583.54  955  
35000   6.40  0.02  583.54  955  
40000   6.34  0.02  583.54  955  
45000   6.33  0.02  583.54  955  
50000   6.51  0.01  584.21  961  
55000   6.48  0.02  584.21  961  
60000   6.33  0.01  584.21  961  
65000   6.33  0.01  584.21  961  
70000   6.26  0.01  584.21  961  
75000   6.36  0.01  584.21  961  
80000   6.25  0.01  584.21  961  
85000   6.33  0.01  584.21  961  
90000   6.38  0.01  584.21  961  
95000   6.36  0.01  584.21  961  
100000  6.39  0.01  584.21  961  
105000  6.48  0.01  584.21  961  
110000  6.47  0.01  584.21  961  
115000  6.40  0.01  584.21  961  
120000  6.36  0.01  584.21  961  
125000  6.46  0.01  584.21  961  
130000  6.47  0.01  584.21  961  
135000  6.51  0.01  584.21  961  
140000  6.64  0.01  584.21  961

0 use file get drive program write window system work one
1 write get article go like think know one say see
2 one people write say think make know post article point
3 god say one people believe christian jesus write know make
4 key space use encryption launch chip system government clipper nasa
5 car bike dod ride get one engine good article write
6 use ground wire circuit power one current amp wiring outlet
7 game year team win player run hit play last season
8 team game player play hockey season go nhl win year
9 pt game play team period la hockey vs goal van
